https://deap.readthedocs.io/en/master/tutorials/basic/part2.html

In [3]:
import random

from deap import base
from deap import creator
from deap import tools

IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [5]:
toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                toolbox.attr_float, n=IND_SIZE)

In [11]:
ind1 = toolbox.individual()
print(ind1, ind1.fitness.values, ind1.fitness.valid)
# fitness.values が空ならば、validがFalse。
# なるほど、評価できるかできないかでvalidの判定か。
# なら、評価関数が遺伝子の妥当性検証をしているということか

print(type(ind1))
print(dir(ind1))

[0.762544364109863, 0.9409068350101778, 0.8377895059030583, 0.5324252325671645, 0.8075305776371624] () False
<class 'deap.creator.Individual'>
['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'append', 'clear', 'copy', 'count', 'extend', 'fitness', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [15]:
# 評価処理は自分で書く必要がある
# > The evaluation is the most personal part of an evolutionary algorithm, 
# > it is the only part of the library that you must write yourself. 

# 評価値の典型例が、タプルと書いてある。1つの値ではないのか
# > A typical evaluation function takes one individual as argument 
# > and returns its fitness as a tuple.
# そうか、多目的最適化の場合があるか。それだったら評価値が複数の場合もある
# だが、それらをまとめて1つの評価値にすることもある。
# だが、パレート最適を求めるから、複数の評価値を返すのがデフォルトのほうが勝手が良いな。

def evaluate(individual):
    a = sum(individual)
    b = len(individual)
    return a, 1./b

ind1.fitness.values = evaluate(ind1)
print(ind1.fitness.valid, ind1.fitness, ind1.fitness.values)

True (3.881196515227426, 0.2) (3.881196515227426, 0.2)


In [26]:
# 突然変異は、個体に対して適用するが、当然個体の遺伝子を上書きする。
# つまり、子を作ってから突然変異適用しないと、親が改変されてしまう
# The general rule for mutation operators is that they only mutate, 
# this means that an independent copy must be made prior to mutating the individual 
# if the original individual has to be kept or is a reference to another individual 
# (see the selection operator).

# mutateは遺伝子全部にmutateするのか。遺伝子の一部だけじゃなく。
# つまり、突然変異が起こると、全遺伝子が改変されているってことか？これはおかしい
# 全くの別の個体になるじゃないか。
# 突然変異が強すぎる。
# いや、大丈夫だ。par indpbが各遺伝子に適用するかどうかの確率だな。

mutant = toolbox.clone(ind1)
ind2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.2)
del mutant.fitness.values

print(ind2 is mutant, mutant is ind1)
print(ind1, '\n', ind2, '\n', mutant)

True False
[0.762544364109863, 0.9409068350101778, 0.8377895059030583, 0.5324252325671645, 0.8075305776371624] 
 [0.762544364109863, 0.9409068350101778, 0.8377895059030583, 0.30604731181510525, 0.8075305776371624] 
 [0.762544364109863, 0.9409068350101778, 0.8377895059030583, 0.30604731181510525, 0.8075305776371624]


In [28]:
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxBlend(child1, child2, 0.5)
del child1.fitness.values
del child2.fitness.values

In [29]:
selected = tools.selBest([child1, child2], 2)
print(child1 in selected)

# selectionはただ選ぶだけで、個体の複製はしない
# 最低限の機能しかモジュール側は提供しないんだな
# 複製はユーザー任せ。
# deapは細かいことは自由にできるが、手間が減らせる量が少ないのか。



True
